In [1]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import pickle

In [2]:
!ls

README.md	     generator.py      shmup-train.py	training_1_5.log
__pycache__	     meta_evolve.sh    train.csv	training_1_5_2.log
ai_model.ipynb	     score_utils.py    train_1_5.csv	training_1_6.log
ai_model_forest.pkl  score_utils.pyc   train_1_6.csv	training_1_6_2.log
ai_model_mlp.pkl     scorer.py	       train_1_6_2.csv
evolution	     shmup-play-ai.py  training.log
evolve.sh	     shmup-play.py     training2.log


In [1]:
!cat train.csv | wc -l

840


In [3]:
features = ["f" + str(i) for i in range(0,107)]
label = ["action"]

headers = features + label
print(len(headers))

df = pd.read_csv('train.csv', 
                 sep = ',', 
                 header = None,
                 names = headers)
len(df)

108


,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f98,f99,f100,f101,f102,f103,f104,f105,f106,action
0,0,0,0,0,0,0,0,0,0,0,...,696,-1,4,692,-1,5,647,2,7,4
1,0,0,0,0,0,0,0,0,0,0,...,691,-1,4,687,-1,5,640,2,7,4
2,0,0,0,0,0,0,0,0,0,0,...,685,-1,4,680,-1,5,633,2,7,1
3,0,240,636,-1,1,694,-3,7,665,-1,...,679,-1,4,674,-1,5,627,2,7,1
4,0,240,635,-1,1,688,-3,7,663,-1,...,674,-1,4,667,-1,5,621,2,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,8,89,456,2,2,96,-1,2,574,0,...,638,0,1,649,1,1,605,-1,1,2
836,8,89,455,2,2,94,-1,2,573,0,...,637,0,1,648,1,1,604,-1,1,5
837,8,89,454,2,2,91,-1,2,572,0,...,637,0,1,650,1,1,606,-1,1,0
838,8,89,453,2,2,89,-1,2,571,0,...,636,0,1,650,1,1,605,-1,1,5


In [4]:
len(df)

840

In [ ]:
# import pandas_profiling
# pandas_profiling.ProfileReport(df)

Basic logisting regresion

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25)

classifier = LogisticRegression().fit(X_train, y_train)
y_pred_test = classifier.predict(X_test)
y_pred_train = classifier.predict(X_train)

test_accuracy = accuracy_score(y_test, y_pred_test)
train_accuracy = accuracy_score(y_train, y_pred_train)
print('Training accuracy: {0:.2f}, evaluation accuracy: {1:.2f}'.format(train_accuracy, test_accuracy))

/anaconda3/envs/letsplay/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/envs/letsplay/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Training accuracy: 0.75, evaluation accuracy: 0.55


/anaconda3/envs/letsplay/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


xgboost

In [6]:
y_train = df["action"]
X_train = df.drop(columns="action")

model = XGBClassifier().fit(X_train, y_train)

model.evals_result

<bound method XGBClassifier.evals_result of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)>

random forest model

In [9]:
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier(random_state=1, n_estimators=1000)
forest_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [10]:
pickle.dump(forest_model, open("ai_model_forest.pkl","wb"))

In [11]:
from sklearn.neural_network import MLPClassifier
mlp_model = MLPClassifier(solver='lbfgs', alpha=1e-5, 
                    hidden_layer_sizes=(128, 128, 128), random_state=1)

mlp_model.fit(X_train, y_train)   

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(128, 128, 128), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [12]:
pickle.dump(mlp_model, open("ai_model_mlp.pkl","wb"))

In [ ]:
# y_train = df[0]
# X_train = df.drop(columns=[0])
# y_train_matrix = y_train.values
# X_train_matrix = X_train.values

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns="action"), df["action"], 
                                                    test_size=0.2, random_state=7)

xgb_model = XGBClassifier(n_estimators=1000).fit(X_train, y_train,eval_set = [[X_test, y_test]])

[0]	validation_0-merror:0.369048
[1]	validation_0-merror:0.357143
[2]	validation_0-merror:0.35119
[3]	validation_0-merror:0.357143
[4]	validation_0-merror:0.363095
[5]	validation_0-merror:0.363095
[6]	validation_0-merror:0.345238
[7]	validation_0-merror:0.345238
[8]	validation_0-merror:0.345238
[9]	validation_0-merror:0.345238
[10]	validation_0-merror:0.339286
[11]	validation_0-merror:0.339286
[12]	validation_0-merror:0.339286
[13]	validation_0-merror:0.339286
[14]	validation_0-merror:0.333333
[15]	validation_0-merror:0.333333
[16]	validation_0-merror:0.333333
[17]	validation_0-merror:0.333333
[18]	validation_0-merror:0.339286
[19]	validation_0-merror:0.339286
[20]	validation_0-merror:0.333333
[21]	validation_0-merror:0.333333
[22]	validation_0-merror:0.333333
[23]	validation_0-merror:0.333333
[24]	validation_0-merror:0.327381
[25]	validation_0-merror:0.327381
[26]	validation_0-merror:0.327381
[27]	validation_0-merror:0.333333
[28]	validation_0-merror:0.339286
[29]	validation_0-merror:

[238]	validation_0-merror:0.363095
[239]	validation_0-merror:0.369048
[240]	validation_0-merror:0.369048
[241]	validation_0-merror:0.369048
[242]	validation_0-merror:0.369048
[243]	validation_0-merror:0.369048
[244]	validation_0-merror:0.369048
[245]	validation_0-merror:0.369048
[246]	validation_0-merror:0.369048
[247]	validation_0-merror:0.369048
[248]	validation_0-merror:0.369048
[249]	validation_0-merror:0.369048
[250]	validation_0-merror:0.363095
[251]	validation_0-merror:0.369048
[252]	validation_0-merror:0.369048
[253]	validation_0-merror:0.375
[254]	validation_0-merror:0.375
[255]	validation_0-merror:0.375
[256]	validation_0-merror:0.375
[257]	validation_0-merror:0.375
[258]	validation_0-merror:0.375
[259]	validation_0-merror:0.369048
[260]	validation_0-merror:0.375
[261]	validation_0-merror:0.375
[262]	validation_0-merror:0.369048
[263]	validation_0-merror:0.369048
[264]	validation_0-merror:0.369048
[265]	validation_0-merror:0.369048
[266]	validation_0-merror:0.369048
[267]	val

[476]	validation_0-merror:0.375
[477]	validation_0-merror:0.369048
[478]	validation_0-merror:0.363095
[479]	validation_0-merror:0.363095
[480]	validation_0-merror:0.363095
[481]	validation_0-merror:0.380952
[482]	validation_0-merror:0.375
[483]	validation_0-merror:0.363095
[484]	validation_0-merror:0.363095
[485]	validation_0-merror:0.369048
[486]	validation_0-merror:0.369048
[487]	validation_0-merror:0.369048
[488]	validation_0-merror:0.369048
[489]	validation_0-merror:0.363095
[490]	validation_0-merror:0.369048
[491]	validation_0-merror:0.363095
[492]	validation_0-merror:0.369048
[493]	validation_0-merror:0.380952
[494]	validation_0-merror:0.375
[495]	validation_0-merror:0.375
[496]	validation_0-merror:0.380952
[497]	validation_0-merror:0.386905
[498]	validation_0-merror:0.386905
[499]	validation_0-merror:0.380952
[500]	validation_0-merror:0.380952
[501]	validation_0-merror:0.386905
[502]	validation_0-merror:0.386905
[503]	validation_0-merror:0.386905
[504]	validation_0-merror:0.3869

[715]	validation_0-merror:0.363095
[716]	validation_0-merror:0.363095
[717]	validation_0-merror:0.363095
[718]	validation_0-merror:0.363095
[719]	validation_0-merror:0.363095
[720]	validation_0-merror:0.363095
[721]	validation_0-merror:0.363095
[722]	validation_0-merror:0.363095
[723]	validation_0-merror:0.363095
[724]	validation_0-merror:0.363095
[725]	validation_0-merror:0.363095
[726]	validation_0-merror:0.363095
[727]	validation_0-merror:0.363095
[728]	validation_0-merror:0.363095
[729]	validation_0-merror:0.363095
[730]	validation_0-merror:0.363095
[731]	validation_0-merror:0.363095
[732]	validation_0-merror:0.363095
[733]	validation_0-merror:0.363095
[734]	validation_0-merror:0.363095
[735]	validation_0-merror:0.363095
[736]	validation_0-merror:0.363095
[737]	validation_0-merror:0.363095
[738]	validation_0-merror:0.363095
[739]	validation_0-merror:0.363095
[740]	validation_0-merror:0.363095
[741]	validation_0-merror:0.363095
[742]	validation_0-merror:0.363095
[743]	validation_0-m

[959]	validation_0-merror:0.375
[960]	validation_0-merror:0.375
[961]	validation_0-merror:0.375
[962]	validation_0-merror:0.375
[963]	validation_0-merror:0.375
[964]	validation_0-merror:0.375
[965]	validation_0-merror:0.375
[966]	validation_0-merror:0.375
[967]	validation_0-merror:0.375
[968]	validation_0-merror:0.375
[969]	validation_0-merror:0.375
[970]	validation_0-merror:0.375
[971]	validation_0-merror:0.375
[972]	validation_0-merror:0.375
[973]	validation_0-merror:0.375
[974]	validation_0-merror:0.375
[975]	validation_0-merror:0.375
[976]	validation_0-merror:0.375
[977]	validation_0-merror:0.375
[978]	validation_0-merror:0.375
[979]	validation_0-merror:0.375
[980]	validation_0-merror:0.375
[981]	validation_0-merror:0.375
[982]	validation_0-merror:0.375
[983]	validation_0-merror:0.375
[984]	validation_0-merror:0.375
[985]	validation_0-merror:0.375
[986]	validation_0-merror:0.369048
[987]	validation_0-merror:0.369048
[988]	validation_0-merror:0.369048
[989]	validation_0-merror:0.369

In [ ]:
pickle.dump(xgb_model, open("ai_model_xgb.pkl","wb"))

In [ ]:
!ls

In [7]:
# model_p = pickle.load(open("ai_model.pkl","rb"))

model.predict(np.array([8,280,630,-1,1,674,-3,7,667,-1,2,649,-3,2,662,0,3,662,-1,4,655,-1,5,608,2,7,1,8,288,630,-1,1,672,-3,7,668,-1,2,650,-3,2,661,0,3,657,-1,4,649,-1,5,602,2,7,1,8,296,629,-1,1,670,-3,7,669,-1,2,651,-3,2,660,0,3,651,-1,4,643,-1,5,596,2,7,1,8,296,628,-1,1,665,-3,7,667,-1,2,650,-3,2,658,0,3,647,-1,4,638,-1,5,589,2,7]).reshape(1, -1))

array([4])